In [1]:
import http.client, urllib.request, urllib.parse, urllib.error
import json
from xml.etree import ElementTree

In [2]:
yandex_api_key = 'trnsl.1.1.20170310T112707Z.2838d6f86832fe3a.cfb3f9f194a56f93f29da44f892ba1d5be713e2a'
translation_api_key = 'b9e77226876c4d0c8c0b9a8297cea1bb'
token_translation = ''


def send_request(method, host, url, body=None, params=None, headers=None):
    try:
        conn = http.client.HTTPSConnection(host)
        if params is None:
            if body is None:
                conn.request(method, url, headers=headers)
            else:
                conn.request(method, url, body=body, headers=headers)
        else:
            if body is None and headers is None:
                conn.request(method, "%s?%s" % (url, params))
            else:
                conn.request(method, "%s?%s" % (url, params), body=body, headers=headers)
        response = conn.getresponse()
        data = response.read()
        headers = response.getheaders()
        status = response.status
        conn.close()
        return data, headers, status
    except Exception as e:
        print("Error. HTML Response: ")
        print(response.status)
        print(response.read().decode('utf-8'))
        print(response.getheaders())
        raise e


def translate_yandex(text, language):
    params = urllib.parse.urlencode({
        'key': yandex_api_key,
        'text': text,
        'lang': language,
    })

    try:
        data, headers, status = send_request("GET", 'translate.yandex.net', "/api/v1.5/tr.json/translate", params=params)
        json_obj = json.loads(data.decode('utf-8'))
        return json_obj['text'][0]
    except Exception as e:
        raise e


def get_token_translation():
    print("Getting access token for text translation")
    headers = {
        'Ocp-Apim-Subscription-Key': translation_api_key,
        'Content-Length': 0,
    }
    global token_translation

    try:
        data, headers, status = send_request("POST", 'api.cognitive.microsoft.com', "/sts/v1.0/issueToken", headers=headers)
        token_translation = data.decode('utf-8')
        print(token_translation)
    except Exception as e:
        raise e


def get_microsoft_translation(text, language):
    headers = {
        'Authorization': 'Bearer {token}'.format(token=token_translation)
    }

    params = urllib.parse.urlencode({
        'text': text,
        'to': language,
    })

    try:
        data, headers, status = send_request("GET", 'api.microsofttranslator.com', "/v2/http.svc/Translate", headers=headers, params=params)
        tree = ElementTree.fromstring(data.decode('utf-8'))
        return "".join((tree.text.split('\n')[0].rsplit('\xa0')))
    except Exception as e:
        raise e

In [3]:
def test_translator_metric(original_tr, mutants_tr):
    score = 0
    for m in mutants_tr:
        if m == original_tr:
            score += 1
    return score/len(mutants_tr)

# test_translator_metric('Quel est votre nom?', ['Quel est votre nom?', 'Quel est le nom de votre?', 'Quel est votre nom?', 'Quel est le nom de votre est?'])

In [21]:
def test_translator(file, translator):
    general_score = 0
    with open(file, 'r') as f:
        nb_ex = int(f.readline().split('\n')[0])
        for ex in range(nb_ex):
            nb_mutants = int(f.readline().split('\n')[0])
            original = f.readline().split('\n')[0]
            original_tr = translator(original, "fr")
            mutants = []
            mutants_tr = []

            for n in range(nb_mutants):
                m = f.readline().split('\n')[0]
                mutants.append(m)
                mutants_tr.append(translator(m, "fr"))
            
            score = test_translator_metric(original_tr, mutants_tr)
            general_score += score
            print("The robustness rate is: {}".format(score))
            print(original_tr, mutants_tr)
        f.close()
        print("The average robustness on this test case is of: {}".format(general_score/nb_ex))
    

In [18]:
get_token_translation()

Getting access token for text translation
eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJzY29wZSI6Imh0dHBzOi8vYXBpLm1pY3Jvc29mdHRyYW5zbGF0b3IuY29tLyIsInN1YnNjcmlwdGlvbi1pZCI6Ijk3ODYxNmE2MzgzMTQzM2VhN2NmZjVhOWMwM2YxNTU5IiwicHJvZHVjdC1pZCI6IlRleHRUcmFuc2xhdG9yLkYwIiwiY29nbml0aXZlLXNlcnZpY2VzLWVuZHBvaW50IjoiaHR0cHM6Ly9hcGkuY29nbml0aXZlLm1pY3Jvc29mdC5jb20vaW50ZXJuYWwvdjEuMC8iLCJhenVyZS1yZXNvdXJjZS1pZCI6Ii9zdWJzY3JpcHRpb25zL2FhZjVjNjk1LTJiM2ItNDM0Yy1hODBhLWYzOGNjMzgzNDg2NC9yZXNvdXJjZUdyb3Vwcy9ncm91cGUvcHJvdmlkZXJzL01pY3Jvc29mdC5Db2duaXRpdmVTZXJ2aWNlcy9hY2NvdW50cy9UcmFuc2xhdG9yIiwiaXNzIjoidXJuOm1zLmNvZ25pdGl2ZXNlcnZpY2VzIiwiYXVkIjoidXJuOm1zLm1pY3Jvc29mdHRyYW5zbGF0b3IiLCJleHAiOjE0OTI2OTg1OTh9.Li1DxBsOFrf0KjQwiUHIHL3JI_m_HVY4qWfSd57SGGI


In [24]:
test_translator('ireland_small.txt_mut_homophones', translate_yandex)

The robustness rate is: 0.0
en europe, l'irlande est une île de l'atlantique nord ["auberge de l'europe, l'irlande est une île de l'atlantique nord", "auberge de l'europe, l'irlande est une île de l'atlantique nord"]
The robustness rate is: 0.14285714285714285
elle est séparée de la grande-bretagne à l'est par le canal du nord de la mer d'irlande et le canal saint-georges ["elle est séparée de la grande-bretagne à l'est d'acheter le chenal nord la mer d'irlande et le canal saint-georges", "elle est séparée de la grande-bretagne à l'est, au revoir le chenal nord la mer d'irlande et le canal saint-georges", "elle est séparée de la grille de la grande-bretagne à l'est par le canal du nord de la mer d'irlande et le canal saint-georges", "elle est séparée de la grande-bretagne à l'est par le canal du nord de la mer d'irlande et le canal saint-georges", "elle est séparée de la grande-bretagne à l'est par le canal du nord, les irlandais voir et le canal saint-georges", "elle est séparée de la

In [19]:
test_translator('test2.txt', get_microsoft_translation)

The robustness rate is: 0.0
Comment vas-tu? ['Comment arg vous?', 'Comment les aou?', 'Comment les ygu?']
The robustness rate is: 0.0
Comment vous appelez-vous? ['Whwt est votre nom?', 'Whdt est votre nom?', 'Quel ih votre nom?']
The robustness rate is: 0.0
C’est l’été ['C’est majda', 'C’est gummer', 'C’est sumfer']
The average robustness on this test case is of: 0.0


Conclusion here:
* Some translations resist to mutation (give the same answer)
* Some translations dont't resist (different answer) but give an answer with good semantic
* Some are totally bad

We should:
* Be able to consider a different answer with similar semantic as a good result
* Give % of robustness
* Give aspects in which the robot is not robust

So, ways of development:
* New bots. To test robustness of other types of bot.
* New mutant types. To test robustness to different errors.
* Method to compare the semantic similarity. To be sure that the created mutants mean the same thing.